In [4]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib as plt
from collections import defaultdict
import math
import matplotlib.pyplot as plt

In [5]:
def get_data(portfolio, start_date, end_date):
    # Initialize an empty DataFrame
    data = pd.DataFrame()

    # Loop through each country in the portfolio
    for tickers in portfolio:
        # Download data for each ticker
        df = yf.download(tickers, start=start_date, end=end_date)

        # Concatenate the data to the main DataFrame
        data = pd.concat([data, df], axis=1)

    return data

In [6]:
START_DATE = "1984-01-01"
END_DATE = "2024-04-26"
columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'] # total data
features = ['Open', 'Close', 'High', 'Low', 'Volume'] # input features
portfolio = ['^GSPC']

In [7]:
data = get_data(portfolio, start_date=START_DATE, end_date=END_DATE)
data.interpolate(method = 'linear', inplace = True,limit_direction = 'both')

[*********************100%%**********************]  1 of 1 completed


In [9]:
test_data = data
CPIdf = pd.read_csv("MEDCPIM158SFRBCLE.csv")

,Open,High,Low,Close,Adj Close,Volume,CPI,Adjusted Price
Date,,,,,,,,
1984-01-03,164.929993,164.929993,163.979996,164.039993,164.039993,71340000,2,82.019997
1984-01-04,164.089996,166.779999,164.039993,166.779999,166.779999,112980000,2,83.389999
1984-01-05,166.779999,169.100006,166.779999,168.809998,168.809998,159990000,2,84.404999
1984-01-06,168.809998,169.309998,168.490005,169.279999,169.279999,137590000,2,84.639999
1984-01-09,169.179993,169.460007,168.479996,168.899994,168.899994,107100000,2,84.449997
...,...,...,...,...,...,...,...,...
2024-04-19,5005.439941,5019.020020,4953.560059,4967.229980,4967.229980,3878750000,2,2483.614990
2024-04-22,4987.330078,5038.839844,4969.399902,5010.600098,5010.600098,3820250000,2,2505.300049
2024-04-23,5028.850098,5076.120117,5027.959961,5070.549805,5070.549805,3751400000,2,2535.274902


In [10]:
print(CPIdf)

           DATE  MEDCPIM158SFRBCLE
0    1983-01-01           5.014653
1    1983-02-01           4.952545
2    1983-03-01           3.612261
3    1983-04-01           4.828054
4    1983-05-01           3.767326
..          ...                ...
490  2023-11-01           5.122615
491  2023-12-01           4.160125
492  2024-01-01           6.542477
493  2024-02-01           4.558160
494  2024-03-01           4.327346

[495 rows x 2 columns]


In [ ]:
CPIdf['DATE'] = pd.to_datetime(CPIdf['DATE'], format='%Y-%m-%d')
CPIDf = CPIdf.set_index('DATE', inplace=True, drop=True)

In [31]:
CPIdf
dates = CPIdf.resample('D').interpolate('pad')

C:\Users\tanis\AppData\Local\Temp\ipykernel_31708\1129445586.py:2: FutureWarning: DataFrame.interpolate with method=pad is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dates = CPIdf.resample('D').interpolate('pad')


In [32]:
print(dates)

            MEDCPIM158SFRBCLE
DATE                         
1983-01-01           5.014653
1983-01-02           5.014653
1983-01-03           5.014653
1983-01-04           5.014653
1983-01-05           5.014653
...                       ...
2024-02-26           4.558160
2024-02-27           4.558160
2024-02-28           4.558160
2024-02-29           4.558160
2024-03-01           4.327346

[15036 rows x 1 columns]


In [44]:
test_data['CPI'] = dates['MEDCPIM158SFRBCLE']
base_cpi = dates['MEDCPIM158SFRBCLE']['1984-01-03']
test_data
test_data['Adjusted Price'] = test_data['Close'] * (base_cpi / test_data['CPI'])
test_data

,Open,High,Low,Close,Adj Close,Volume,CPI,Adjusted Price,True CPI
Date,,,,,,,,,
1984-01-03,164.929993,164.929993,163.979996,164.039993,164.039993,71340000,4.85249,164.039993,104.852490
1984-01-04,164.089996,166.779999,164.039993,166.779999,166.779999,112980000,4.85249,166.779999,109.940448
1984-01-05,166.779999,169.100006,166.779999,168.809998,168.809998,159990000,4.85249,168.809998,115.275297
1984-01-06,168.809998,169.309998,168.490005,169.279999,169.279999,137590000,4.85249,169.279999,120.869020
1984-01-09,169.179993,169.460007,168.479996,168.899994,168.899994,107100000,4.85249,168.899994,126.734178
...,...,...,...,...,...,...,...,...,...
2024-04-19,5005.439941,5019.020020,4953.560059,4967.229980,4967.229980,3878750000,NaN,NaN,NaN
2024-04-22,4987.330078,5038.839844,4969.399902,5010.600098,5010.600098,3820250000,NaN,NaN,NaN
2024-04-23,5028.850098,5076.120117,5027.959961,5070.549805,5070.549805,3751400000,NaN,NaN,NaN


In [45]:
test_data[(test_data.index.year >= 2021) & (test_data.index.year <= 2023)]

,Open,High,Low,Close,Adj Close,Volume,CPI,Adjusted Price,True CPI
Date,,,,,,,,,
2021-01-04,3764.610107,3769.989990,3662.709961,3700.649902,3700.649902,5015000000,1.171262,15331.641858,6.930561e+115
2021-01-05,3698.020020,3737.830078,3695.070068,3726.860107,3726.860107,4591020000,1.171262,15440.229670,7.011736e+115
2021-01-06,3712.199951,3783.040039,3705.340088,3748.139893,3748.139893,6064110000,1.171262,15528.390953,7.093862e+115
2021-01-07,3764.709961,3811.550049,3764.709961,3803.790039,3803.790039,5099160000,1.171262,15758.947244,7.176949e+115
2021-01-08,3815.050049,3826.689941,3783.600098,3824.679932,3824.679932,4773040000,1.171262,15845.493219,7.261010e+115
...,...,...,...,...,...,...,...,...,...
2023-12-22,4753.919922,4772.939941,4736.770020,4754.629883,4754.629883,3046770000,4.160125,5545.937994,2.153762e+132
2023-12-26,4758.859863,4784.720215,4758.450195,4774.750000,4774.750000,2513910000,4.160125,5569.406682,2.243361e+132
2023-12-27,4773.450195,4785.390137,4768.899902,4781.580078,4781.580078,2748450000,4.160125,5577.373483,2.336688e+132


In [ ]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1984-01-03,164.929993,164.929993,163.979996,164.039993,164.039993,71340000
1984-01-04,164.089996,166.779999,164.039993,166.779999,166.779999,112980000
1984-01-05,166.779999,169.100006,166.779999,168.809998,168.809998,159990000
1984-01-06,168.809998,169.309998,168.490005,169.279999,169.279999,137590000
1984-01-09,169.179993,169.460007,168.479996,168.899994,168.899994,107100000
...,...,...,...,...,...,...
2024-04-19,5005.439941,5019.020020,4953.560059,4967.229980,4967.229980,3878750000
2024-04-22,4987.330078,5038.839844,4969.399902,5010.600098,5010.600098,3820250000
2024-04-23,5028.850098,5076.120117,5027.959961,5070.549805,5070.549805,3751400000


In [ ]:
jan_to_april = data[(data.index.month >= 1) & (data.index.month <= 4)]
july_to_oct = data[(data.index.month >= 7) & (data.index.month <= 10)]

In [ ]:
def filter_by_yr(df, start_yr, end_yr):
	dfs_by_year = {}

	for year in range(start_yr, end_yr + 1):
		dfs_by_year[year] = df[df.index.year == year]

	return dfs_by_year

In [ ]:
jan_to_april_by_yr = filter_by_yr(jan_to_april, 1984, 2024)
july_to_oct_by_yr = filter_by_yr(july_to_oct, 1984, 2024)

In [ ]:
def generate_labels(df, start_yr, end_yr):
	labels = []

	for year in range(start_yr, end_yr + 1):
		curr = df[year]
		diff = curr["Adj Close"].iloc[-1] - curr["Adj Close"].iloc[0]
		labels.append(1 if diff > 0 else 0)

	return labels

In [ ]:
labels = generate_labels(july_to_oct_by_yr, 1984, 2023)

In [ ]:
jan_to_april_by_yr[2022]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,4778.140137,4796.640137,4758.169922,4796.560059,4796.560059,3831020000
2022-01-04,4804.509766,4818.620117,4774.270020,4793.540039,4793.540039,4683170000
2022-01-05,4787.990234,4797.700195,4699.439941,4700.580078,4700.580078,4887960000
2022-01-06,4693.390137,4725.009766,4671.259766,4696.049805,4696.049805,4295280000
2022-01-07,4697.660156,4707.950195,4662.740234,4677.029785,4677.029785,4181510000
...,...,...,...,...,...,...
2022-04-25,4255.339844,4299.020020,4200.819824,4296.120117,4296.120117,5240040000
2022-04-26,4278.140137,4278.140137,4175.040039,4175.200195,4175.200195,4689970000
2022-04-27,4186.520020,4240.709961,4162.899902,4183.959961,4183.959961,4769680000
